In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np
import random
import pandas as pd

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [9]:
df_data=pd.read_csv('/users/gavin/Documents/Metis/Coursework/Project_3/data_cleaning/mvp_data.csv')
df_clean = pd.read_csv("/users/gavin/Documents/Metis/Coursework/Project_3/data_cleaning/mvp_grid_final.csv")

In [11]:
df_data.drop('Unnamed: 0', axis=1, inplace = True)

In [13]:
df_data.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_fix,team_away_fix,team_favorite_id_fix,target,target_spread,home_away,target_adj_score,opposition_score,result,date_formatted,year_x,offense_rank,team_x,target_games_played,points_scored,yards,target_offensive_plays,target_yards_per_offensive_play,target_offense_turnovers_lost,target_offense_fumbles_lost,target_offense_first_down,target_offense_pass_completed,target_offense_passes_attempted,target_offense_yards_gained_passing,target_offense_passing_touchdowns,target_offense_interceptions_thrown,target_offense_ net_gain_from_passess_and_sacks,target_offense_first_down_passing,target_offense_rusing_attempts,target_offense_rushing_yards,target_offense_rushing_touchdowns,target_offense_rushing_yards_per_attempt,target_offense_first_downs_by_rushing,target_offense_penalties_committed,target_offense_penalty_yards,target_offense_first_downs_from_penalty,target_offense_percent_of_drives_with_offensive_score,offense_percent_drives_ending_in_turnover,offense_extra_points,name_adj_x,Year_x,target_passing_rank,Team_x,target_games_played_x,target_offense_passes_completed,target_offese_passes_attempted,target_offense_percent_pass_completed,target_yards_gained_passing,target_offense_passing_touchdowns.1,target_offense_passing_touchdown_percentage,target_offense_interceptions_thrown.1,target_offense_percent_passes_intercepted,target_offense_longest_completed_pass,target_offense_yards_gained_per_pass_attempt,target_offense_adjusted_yards_gained_per_pass_attempt,target_offense_yards_gained_per_pass_completion,target_offense_yards_gained_per_game_played,target_offense_quarterback_rating,target_offense_times_sacked,target_offense_yards_lost_due_to_sacks,target_offense_NY/A,target_offense_net_yards_gained_per_pass_attempt,target_offense_percentage_of_times_sacked_while_passing,target_offense_comebacks_led_by_QB,target_offense_game_winning_drives_by_qb,target_offense_expected_points_contributed_by_passing_offense,target_name_adj,Year_y,target_offenseive_rushing_Rank,team_y,target_games_played_y,target_offensive_rushing_attempts,target_offensive_rushing_yards_gained,target_offensive_rushing_touchdowns,target_offensive_longest_rushing_attempt,target_offensive_rushing_Y/A,target_offensive_rushing_Y/G,target_offensive_rushing_Fmb,target_offensive_rushing_EXP,name_adj_y,Year_x.1,target_defense_rank,Team_y,games_played,target_defense_points_scored_by_team,target_defense)_yards_per_game,target_defense_offensive_plays,target_defense_yards_per_offensive_plays,target_defense_takeaways,target_defense_fumbles_lost_by_team,target_defense_1stD,target_defense_passes_completed,target_defense_passes_attempted,target_defense_yards_gained_by_passing,target_defense_passing_touchdown,target_defense_interceptions_thrown,target_defense_net_yards_gained_per_pass_attempt,target_defense_first_downs_by_passing,target_defense_rushing_attempts,target_defense_rushing_yards_gained,target_defense_rushing_tds,target_defense_rushing_yards_per_attempt,target_defense_first_downs_by_rushing,target_defense_penalties_commited_by_team,target_defense_penalties_commited_by_team_yards,target_defense_first_downs_by_penalty,target_defense_percent_of_drives_ending_in_offensive_score,target_defense_percentage_of_drives_ending_in_offensive_turnover,target_defense_expected_points_contributed_by_all_defense,target_defense_name_adj,opponent,year_y,opponent_offense_rank,opponent_team,opponent_games_played,opponent_points_scored,opponent_yards,opponent_offensive_plays,opponent_yards_per_offensive_play,opponent_offense_turnovers_lost,opponent_offense_fumbles_lost,opponent_offense_first_down,opponent_offense_pass_completed,opponent_offense_passes_attempted,opponent_offense_yards_gained_passing,opponent_offense_passing_touchdowns,opponent_offense_inter

In [87]:
df_data_fix = df_data.drop(['schedule_date','schedule_week', 'team_home', 'team_away', 'team_favorite_id', 'stadium', 'team_home_fix',  'team_away_fix','team_favorite_id_fix','target', 'date_formatted', 'year_x', 'team_x', 'name_adj_x', 'Year_x', 'Team_x', 'target_games_played_x', 'target_name_adj', 'Year_y', 'team_y', 'target_games_played_y', 'name_adj_y', 'Year_x.1', 'Team_y', 'games_played', 'target_defense_name_adj', 'opponent', 'year_y', 'opponent_team', 'opponent_games_played', 'opponent_name_adj', 'Year_y.1', 'opponent_defense_Team', 'opponent_defense_Team', 'opponent_defense_name_adj', 'weather_detail', 'target_offensive_rushing_EXP', 'weather_humidity', 'weather_temperature', 'weather_wind_mph', 'target_offense_game_winning_drives_by_qb', 'target_offense_comebacks_led_by_QB', 'opposition_score', 'target_adj_score', 'score_home', 'score_away', 'opponent_points_scored'], axis=1)

In [88]:
df_data_fix['home_away_fix']=0
df_data_fix.loc[df_data_fix['home_away']=='H', 'home_away_fix']=1
df_data_fix.drop('home_away', axis=1, inplace=True)

df_data_fix['result_score'] = 0
df_data_fix.loc[df_data_fix['result']=='W', 'result_score']=1
df_data_fix.drop('result', axis=1, inplace=True)

In [89]:
X = df_data_fix.drop('result_score', axis=1)
y= df_data_fix['result_score']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [244]:
from sklearn.model_selection import train_test_split

def quick_test(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
    model.fit(xtrain, ytrain)
    return model.score(xtest, ytest)

def quick_test_expand(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
    model.fit(xtrain, ytrain)
    return (model.score(xtest, ytest), model.score(xtrain, ytrain))

def quick_test_afew_times(model, X, y, n=10):
    return np.mean([quick_test(model, X, y) for j in range(n)])

def quick_test_afew_times_view(model, X, y, n=10):
    return [quick_test_expand(model, X, y) for j in range(n)]

In [91]:
randomforest = RandomForestClassifier(n_estimators=100)
quick_test_afew_times_view(randomforest, x_train, y_train)

[(0.5008912655971479, 1.0),
 (0.5401069518716578, 0.9992354740061162),
 (0.5026737967914439, 0.9992354740061162),
 (0.5115864527629234, 1.0),
 (0.5080213903743316, 0.9992354740061162),
 (0.5115864527629234, 0.9992354740061162),
 (0.5169340463458111, 1.0),
 (0.5026737967914439, 1.0),
 (0.49554367201426025, 1.0),
 (0.5151515151515151, 0.9992354740061162)]

In [92]:
important = list(zip(randomforest.feature_importances_, df_data_fix.columns))

In [93]:
important.sort(reverse=True)
important

[(0.1078770768818649, 'over_under_line'),
 (0.09948029790548932, 'target_spread'),
 (0.08679737535464487, 'spread_favorite'),
 (0.009393758230047462, 'opponent_defense_penalties_commited_by_team_yards'),
 (0.00911635825013383, 'opponent_offensive_plays'),
 (0.008734857699139776, 'opponent_defense_passes_attempted'),
 (0.008653173894976112, 'opponent_offense_penalty_yards'),
 (0.008188451541541446, 'opponent_defense_first_downs_by_rushing'),
 (0.008037500242265062, 'opponent_offense_yards_gained_passing'),
 (0.007890987506606016, 'opponent_defense_passes_completed'),
 (0.007826271768842909, 'opponent_offense_passes_attempted'),
 (0.007753689693894156, 'opponent_defense_rushing_attempts'),
 (0.0077468299048711895, 'opponent_offense_rank'),
 (0.007736691129869918, 'opponent_offense_rusing_attempts'),
 (0.007662914991994171, 'opponent_defense_penalties_commited_by_team'),
 (0.007598300722118612, 'opponent_defense_offensive_plays'),
 (0.007586003957218185, 'opponent_defense_yards_per_game')

In [55]:
df

schedule_season                                                         0
schedule_playoff                                                        0
score_home                                                              0
score_away                                                              0
spread_favorite                                                         0
over_under_line                                                         0
stadium_neutral                                                         0
weather_temperature                                                   418
weather_wind_mph                                                      418
target_spread                                                           0
target_adj_score                                                        0
opposition_score                                                        0
offense_rank                                                            0
target_games_played                   

# XGBoost to adjust for overfitting

In [95]:
# see https://ipython.readthedocs.io/en/stable/interactive/magics.html
%pylab inline

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']

#######################
#       imports       #
#######################
import pandas as pd
import seaborn as sns
from sklearn.datasets import california_housing
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
import xgboost as xgb

sns.set_style("whitegrid")

Populating the interactive namespace from numpy and matplotlib


In [96]:
df1=df_data_fix.copy()

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

X = df1.drop('result_score', axis=1)
y= df1['result_score']


#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

#Evaluate models with Root Mean Squared Error
def rmse(actuals, preds):
    return np.sqrt(((actuals - preds) ** 2).mean())

In [98]:
#Linear Regression Benchmark:

lr = LinearRegression()
lr.fit(X_train, y_train)
rmse(lr.predict(X_test), y_test)

0.512251599233911

In [99]:
#Random Forest Benchmark

rf = RandomForestRegressor(n_estimators = 100, max_features = 3, n_jobs=-1)
rf.fit(X_train,y_train)
rmse(rf.predict(X_test), y_test)

0.5429915941879011

In [121]:
gbm = xgb.XGBRegressor( 
                       n_estimators=30000, #arbitrary large number
                       max_depth=3,
                       objective="reg:linear",
                       learning_rate=.1, 
                       subsample=1,
                       min_child_weight=1,
                       colsample_bytree=.8
                      )

eval_set=[(X_train,y_train),(X_val,y_val)] #tracking train/validation error as we go
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='rmse',
#                     eval_metric='r2'
                    early_stopping_rounds=20,
                    verbose=True #gives output log as below
                   )

[19:38:03] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.49778	validation_1-rmse:0.49934
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:0.49544	validation_1-rmse:0.49863
[2]	validation_0-rmse:0.49312	validation_1-rmse:0.49803
[3]	validation_0-rmse:0.49119	validation_1-rmse:0.49716
[4]	validation_0-rmse:0.48936	validation_1-rmse:0.49702
[5]	validation_0-rmse:0.48755	validation_1-rmse:0.49725
[6]	validation_0-rmse:0.48591	validation_1-rmse:0.49675
[7]	validation_0-rmse:0.48443	validation_1-rmse:0.49693
[8]	validation_0-rmse:0.48314	validation_1-rmse:0.49710
[9]	validation_0-rmse:0.48170	validation_1-rmse:0.49726
[10]	validation_0-rmse:0.48036	validation_1-rmse:0.49801
[11]	validation_0-rmse:0.47917	validation_1-rmse:0.49767
[12]	validation_0-

# Feature Engineering

In [131]:
df_adjust = pd.read_csv('/Users/gavin/Documents/Metis/Coursework/Project_3/data_cleaning/feature_engineering.csv')
df_adjust = df_adjust.drop('Unnamed: 0', axis=1)


In [172]:
df_adjust.head()

,over_under_line,offensive_defensive_rank_diff,offensive_offensive_rank_dif,offense_defense_passing_completion,offense_rushing_vs_defense_rushing_allowed_yards,defense_offense_rank_dif,QB_rating,beat_spread,target_spread,first_down_vs_first_down_allowed,first_down_allowed_vs_first_down,drives_with_score_vs_drives_against_with_score,drives_against_with_score_vs_drives_with_score
0,51.0,8,2,-0.001416,0.964683,-5,101.9,0,-7.0,1.064220,0.915408,1.337461,0.713580
1,41.0,-8,-14,-0.055876,1.100013,1,76.3,0,-7.5,0.960245,0.873112,0.965944,0.664198
2,45.5,-15,-21,0.030268,0.909334,-16,94.5,0,-4.0,1.051988,0.903323,1.043344,0.933333
3,48.5,-18,-24,-0.102663,1.199853,-8,74.1,0,-1.0,0.737003,0.966767,0.758514,0.785185
4,41.5,2,-4,0.018954,1.233704,2,95.4,1,-3.5,0.926606,0.945619,1.173375,0.674074


In [136]:
df_adjust['first_down_vs_first_down_allowed'] = df1['target_offense_first_down']/df1['opponent_defense_1stD']

In [137]:
df_adjust['first_down_allowed_vs_first_down'] = df1['target_defense_1stD']/df1['opponent_offense_first_down']

In [764]:
df_adjust['drives_with_score_vs_drives_against_with_score'] = (df1['target_offense_percent_of_drives_with_offensive_score']/df1['opponent_defense_percent_of_drives_ending_in_offensive_score'])/df1['target_spread']

In [139]:
df_adjust['drives_against_with_score_vs_drives_with_score'] = df1['target_defense_percent_of_drives_ending_in_offensive_score']/df1['opponent_offense_percent_of_drives_with_offensive_score']

In [225]:
df_adjust.loc[df_adjust['target_spread']==0, 'spread_to_rank_O_to_D']=df_adjust['offensive_defensive_rank_diff']

In [304]:
df_adjust['yards'] = df1['yards']

In [305]:
df_adjust['is_favorite'] = 0

In [307]:
df_adjust.loc[df_adjust['target_spread']<0, 'is_favorite']=1

In [770]:
df_adjust['target_spread_vs_points'] = (df_adjust['target_spread']*(-1))/df1['points_scored']/df_data['opponent_points_scored']
df_adjust.loc[df_adjust['target_spread']==0, 'drives_with_score_vs_drives_against_with_score']=1

In [772]:
df_adjust[df_adjust['target_spread']==0]

,over_under_line,offensive_defensive_rank_diff,offensive_offensive_rank_dif,offense_defense_passing_completion,offense_rushing_vs_defense_rushing_allowed_yards,defense_offense_rank_dif,QB_rating,beat_spread,target_spread,first_down_vs_first_down_allowed,first_down_allowed_vs_first_down,drives_with_score_vs_drives_against_with_score,drives_against_with_score_vs_drives_with_score,spread_to_rank_O_to_D,yards,is_favorite,target_spread_vs_points
242,49.5,-5,0,0.029531,0.898860,5,104.1,1,-0.0,1.055375,0.947531,1.0,0.729798,-5.0,5728,0,0.0
248,49.5,-3,2,0.031595,1.033325,-4,91.2,0,-0.0,1.078176,1.009259,1.0,0.815657,-3.0,6327,0,0.0
278,45.0,-3,0,0.042488,0.942915,3,92.7,1,-0.0,1.056667,0.946372,1.0,0.939940,-3.0,5353,0,0.0
284,45.0,-4,-1,0.111573,1.036568,10,87.9,0,-0.0,0.990000,1.003155,1.0,1.000000,-4.0,5139,0,0.0
452,43.5,-17,-10,-0.040878,1.158088,5,76.6,0,-0.0,0.916418,0.883008,1.0,1.063768,-17.0,5361,0,0.0
454,46.5,-6,1,-0.115777,1.187912,-5,84.2,1,-0.0,1.002985,0.977716,1.0,1.208696,-6.0,6014,0,0.0
456,43.5,-7,0,0.001045,0.955952,7,87.8,1,-0.0,1.071642,0.933148,1.0,0.994203,-7.0,5985,0,0.0
460,46.5,-7,0,0.001045,0.955952,7,87.8,0,-0.0,1.071642,0.933148,1.0,0.994203,-7.0,5985,0,0.0
718,40.5,-2,0,-0.111894,1.165136,2,67.6,1,-0.0,0.915094,1.092784,1.0,1.269360,-2.0,5268,0,0.0
727,40.5,18,20,-0.052335,1.444737,13,86.7,0,-0.0,1.031447,1.161512,1.0,1.306397,18.0,5666,0,0.0


In [196]:
df_adjust['spread_to_rank_O_to_D'] = (df_adjust['offensive_defensive_rank_diff']*-1)/df_adjust['target_spread']

In [206]:
df_adjust.drop(1418, axis=0, inplace=True)

In [ ]:
df_adjust.loc

In [208]:
df1.drop(1418, axis=0, inplace=True)

In [135]:
df1.columns.to_list()

['schedule_season',
 'schedule_playoff',
 'spread_favorite',
 'over_under_line',
 'stadium_neutral',
 'target_spread',
 'offense_rank',
 'target_games_played',
 'points_scored',
 'yards',
 'target_offensive_plays',
 'target_yards_per_offensive_play',
 'target_offense_turnovers_lost',
 'target_offense_fumbles_lost',
 'target_offense_first_down',
 'target_offense_pass_completed',
 'target_offense_passes_attempted',
 'target_offense_yards_gained_passing',
 'target_offense_passing_touchdowns',
 'target_offense_interceptions_thrown',
 'target_offense_ net_gain_from_passess_and_sacks',
 'target_offense_first_down_passing',
 'target_offense_rusing_attempts',
 'target_offense_rushing_yards',
 'target_offense_rushing_touchdowns',
 'target_offense_rushing_yards_per_attempt',
 'target_offense_first_downs_by_rushing',
 'target_offense_penalties_committed',
 'target_offense_penalty_yards',
 'target_offense_first_downs_from_penalty',
 'target_offense_percent_of_drives_with_offensive_score',
 'offens

In [227]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

X = df_adjust.drop('beat_spread', axis=1)
y= df_adjust['beat_spread']


#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

#Evaluate models with Root Mean Squared Error
def rmse(actuals, preds):
    return np.sqrt(((actuals - preds) ** 2).mean())



In [229]:
df_adjust.isinfinite().sum()

AttributeError: 'DataFrame' object has no attribute 'isinfinite'

In [230]:
#Linear Regression Benchmark:

lr = LinearRegression()
lr.fit(X_train, y_train)
rmse(lr.predict(X_test), y_test)

0.4924514617065663

In [231]:
lg = LogisticRegression()
lg.fit(X_train, y_train)
rmse(lg.predict(X_test), y_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6591334681526583

In [232]:
#Random Forest Benchmark

rf = RandomForestRegressor(n_estimators = 10, max_features = 3, n_jobs=-1)
rf.fit(X_train,y_train)
rmse(rf.predict(X_test), y_test)

0.5393894800938884

In [233]:
gbm = xgb.XGBRegressor( 
                       n_estimators=30000, #arbitrary large number
                       max_depth=3,
                       objective="reg:linear",
                       learning_rate=.1, 
                       subsample=1,
                       min_child_weight=1,
                       colsample_bytree=.8
                      )

eval_set=[(X_train,y_train),(X_val,y_val)] #tracking train/validation error as we go
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='rmse',
                    early_stopping_rounds=20,
                    verbose=True #gives output log as below
                   )

[23:37:19] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.49768	validation_1-rmse:0.49949
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:0.49552	validation_1-rmse:0.49945
[2]	validation_0-rmse:0.49379	validation_1-rmse:0.49871
[3]	validation_0-rmse:0.49219	validation_1-rmse:0.49872
[4]	validation_0-rmse:0.49038	validation_1-rmse:0.49918
[5]	validation_0-rmse:0.48861	validation_1-rmse:0.49882
[6]	validation_0-rmse:0.48729	validation_1-rmse:0.49926
[7]	validation_0-rmse:0.48564	validation_1-rmse:0.49880
[8]	validation_0-rmse:0.48424	validation_1-rmse:0.49851
[9]	validation_0-rmse:0.48315	validation_1-rmse:0.49840
[10]	validation_0-rmse:0.48194	validation_1-rmse:0.49916
[11]	validation_0-rmse:0.48065	validation_1-rmse:0.49913
[12]	validation_0-

In [475]:
gbm = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=3,
                        objective='binary:logistic', #new objective
                        learning_rate=.01, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8
                       )

eval_set=[(X_train,y_train),(X_val,y_val)]
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=50,
                    verbose=True
                   )

print(accuracy_score(y_test, gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit)))

[0]	validation_0-error:0.41349	validation_1-error:0.46629
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 50 rounds.
[1]	validation_0-error:0.40662	validation_1-error:0.47378
[2]	validation_0-error:0.39725	validation_1-error:0.46067
[3]	validation_0-error:0.39788	validation_1-error:0.45506
[4]	validation_0-error:0.40287	validation_1-error:0.46255
[5]	validation_0-error:0.39038	validation_1-error:0.45693
[6]	validation_0-error:0.39101	validation_1-error:0.45880
[7]	validation_0-error:0.39288	validation_1-error:0.44569
[8]	validation_0-error:0.38726	validation_1-error:0.45318
[9]	validation_0-error:0.38601	validation_1-error:0.45693
[10]	validation_0-error:0.39101	validation_1-error:0.45131
[11]	validation_0-error:0.39288	validation_1-error:0.44757
[12]	validation_0-error:0.39038	validation_1-error:0.45506
[13]	validation_0-error:0.38913	validation_1-error:0.45693
[14]	validation_0-error

In [235]:
from __future__ import print_function

In [773]:
X = df_adjust.drop('beat_spread', axis=1)
y= df_adjust['beat_spread']

#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

In [402]:
from sklearn.model_selection import cross_val_score

In [437]:
# 10-fold cross-validation with K=5 for KNN (the n_neighbors parameter)
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
xscale=ss.fit_transform(X)

knn = KNeighborsClassifier(n_neighbors=2)
scores = cross_val_score(knn, xscale, y, cv=10, scoring='accuracy')
print(scores)

[0.58426966 0.53183521 0.60299625 0.52059925 0.52059925 0.50187266
 0.54681648 0.50187266 0.53183521 0.5       ]


In [766]:
randomforest = RandomForestClassifier(n_estimators=100)
quick_test_afew_times_view(randomforest, X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [258]:
def quick_test(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
    model.fit(xtrain, ytrain)
    return model.score(xtest, ytest)

In [265]:
quick_test(randomforest, xtrain, ytrain)

0.5525846702317291

In [267]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

In [268]:
randomforest.fit(xtrain, ytrain)
randomforest.score(xtest, ytest)

0.5243445692883895

In [269]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(ytest,randomforest.predict(xtest))
print(conf_mat)

[[254 157]
 [224 166]]


,over_under_line,offensive_defensive_rank_diff,offensive_offensive_rank_dif,offense_defense_passing_completion,offense_rushing_vs_defense_rushing_allowed_yards,defense_offense_rank_dif,QB_rating,target_spread,first_down_vs_first_down_allowed,first_down_allowed_vs_first_down,drives_with_score_vs_drives_against_with_score,drives_against_with_score_vs_drives_with_score,spread_to_rank_O_to_D
0,51.0,8,2,-0.001416,0.964683,-5,101.9,-7.0,1.064220,0.915408,1.337461,0.713580,1.142857
1,41.0,-8,-14,-0.055876,1.100013,1,76.3,-7.5,0.960245,0.873112,0.965944,0.664198,-1.066667
2,45.5,-15,-21,0.030268,0.909334,-16,94.5,-4.0,1.051988,0.903323,1.043344,0.933333,-3.750000
3,48.5,-18,-24,-0.102663,1.199853,-8,74.1,-1.0,0.737003,0.966767,0.758514,0.785185,-18.000000
4,41.5,2,-4,0.018954,1.233704,2,95.4,-3.5,0.926606,0.945619,1.173375,0.674074,0.571429


In [273]:
df_adjust.corr()

,over_under_line,offensive_defensive_rank_diff,offensive_offensive_rank_dif,offense_defense_passing_completion,offense_rushing_vs_defense_rushing_allowed_yards,defense_offense_rank_dif,QB_rating,beat_spread,target_spread,first_down_vs_first_down_allowed,first_down_allowed_vs_first_down,drives_with_score_vs_drives_against_with_score,drives_against_with_score_vs_drives_with_score,spread_to_rank_O_to_D
over_under_line,1.000000e+00,0.358433,0.015907,0.111643,-0.049665,-0.349411,0.377649,-0.002230,-1.756537e-18,0.147948,-0.144096,0.121797,-0.137879,-0.007775
offensive_defensive_rank_diff,3.584325e-01,1.000000,0.561625,0.100878,0.012770,-0.139875,0.458929,0.094939,-3.325552e-01,0.178460,0.069643,0.198127,0.129486,-0.045403
offensive_offensive_rank_dif,1.590682e-02,0.561625,1.000000,0.205754,0.106182,0.551753,0.390955,0.123361,-4.265890e-01,0.330662,0.343772,0.376893,0.396244,-0.014156
offense_defense_passing_completion,1.116433e-01,0.100878,0.205754,1.000000,0.035144,0.107080,0.587703,0.075552,-2.105024e-01,0.418552,-0.159035,0.571841,-0.160325,-0.041180
offense_rushing_vs_defense_rushing_allowed_yards,-4.966509e-02,0.012770,0.106182,0.035144,1.000000,0.009915,0.121530,0.012256,-7.389553e-02,0.244044,-0.024704,0.285850,-0.048096,0.022880
defense_offense_rank_dif,-3.494114e-01,-0.139875,0.551753,0.107080,0.009915,1.000000,-0.009891,0.079169,-3.030553e-01,0.061866,0.178915,0.130549,0.210966,0.013028
QB_rating,3.776487e-01,0.458929,0.390955,0.587703,0.121530,-0.009891,1.000000,0.115944,-3.438636e-01,0.409948,-0.123196,0.543947,-0.187053,-0.034914
beat_spread,-2.229714e-03,0.094939,0.123361,0.075552,0.012256,0.079169,0.115944,1.000000,1.844444e-02,0.057731,-0.000373,0.095279,-0.004787,-0.025550
target_spread,-1.756537e-18,-0.332555,-0.426589,-0.210502,-0.073896,-0.303055,-0.343864,0.018444,1.000000e+00,-0.284403,0.058941,-0.331500,0.069514,-0.004410
first_down_vs_first_down_allowed,1.479477e-01,0.178460,0.330662,0.418552,0.244044,0.061866,0.409948,0.057731,-2.844027e-01,1.000000,-0.369770,0.729317,-0.215039,-0.009009


In [274]:
df_adjust.columns.tolist()

['over_under_line',
 'offensive_defensive_rank_diff',
 'offensive_offensive_rank_dif',
 'offense_defense_passing_completion',
 'offense_rushing_vs_defense_rushing_allowed_yards',
 'defense_offense_rank_dif',
 'QB_rating',
 'beat_spread',
 'target_spread',
 'first_down_vs_first_down_allowed',
 'first_down_allowed_vs_first_down',
 'drives_with_score_vs_drives_against_with_score',
 'drives_against_with_score_vs_drives_with_score',
 'spread_to_rank_O_to_D']

In [757]:
important = list(zip(randomforest.feature_importances_, X.columns))

In [758]:
important.sort(reverse=True)

In [759]:
important

[(0.10153354774830457, 'target_spread_vs_points'),
 (0.09844280599022247, 'over_under_line'),
 (0.08554533054824967, 'spread_to_rank_O_to_D'),
 (0.08237449952584014, 'target_spread'),
 (0.06978839906735938, 'offense_defense_passing_completion'),
 (0.0657996494969646, 'offense_rushing_vs_defense_rushing_allowed_yards'),
 (0.06243633172190723, 'QB_rating'),
 (0.059360885059691955, 'first_down_vs_first_down_allowed'),
 (0.05670098189310438, 'drives_against_with_score_vs_drives_with_score'),
 (0.05586133245278103, 'yards'),
 (0.05568557159106187, 'first_down_allowed_vs_first_down'),
 (0.05486885770350809, 'drives_with_score_vs_drives_against_with_score'),
 (0.052321826594023416, 'defense_offense_rank_dif'),
 (0.04876782745196808, 'offensive_defensive_rank_diff'),
 (0.042714663790937225, 'offensive_offensive_rank_dif'),
 (0.0077974893640759555, 'is_favorite')]

In [623]:
X=df_adjust.drop('beat_spread', axis=1)
y=df_adjust['beat_spread']

In [357]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

In [358]:
randomforest.fit(xtrain, ytrain)

RandomForestClassifier()

In [359]:
randomforest.score(xtest, ytest)

0.5480649188514357

In [833]:
important = list(zip(randomforestadj.feature_importances_, xtest.columns))

In [834]:
important.sort(reverse=True)
important

[(0.08316236427321241, 'target_spread_vs_points'),
 (0.07303097283689511, 'drives_with_score_vs_drives_against_with_score'),
 (0.07095538673558104, 'offense_defense_passing_completion'),
 (0.07038450705301154, 'offense_rushing_vs_defense_rushing_allowed_yards'),
 (0.0700085993619258, 'QB_rating'),
 (0.06893318982526903, 'target_spread'),
 (0.06844263977938154, 'first_down_vs_first_down_allowed'),
 (0.06719391001568228, 'drives_against_with_score_vs_drives_with_score'),
 (0.06637717916249661, 'first_down_allowed_vs_first_down'),
 (0.06527253515936145, 'yards'),
 (0.061029833678550256, 'spread_to_rank_O_to_D'),
 (0.0608113180008008, 'over_under_line'),
 (0.0579161642712845, 'offensive_defensive_rank_diff'),
 (0.05752051531151271, 'defense_offense_rank_dif'),
 (0.05615486711388013, 'offensive_offensive_rank_dif'),
 (0.0028060174211547804, 'is_favorite')]

In [811]:
X=df_adjust.drop(['beat_spread'] , axis=1)#'offensive_offensive_rank_dif', 'offensive_defensive_rank_diff','defense_offense_rank_dif'
y=df_adjust['beat_spread']

In [762]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5)

In [749]:
randomforest.fit(xtrain, ytrain)

RandomForestClassifier()

In [750]:
randomforest.score(xtest, ytest)


0.5355805243445693

In [614]:
X=df_adjust[['over_under_line','target_spread', 'spread_to_rank_O_to_D', ]]
y=df_adjust['beat_spread']

In [617]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

In [543]:
randomforest.fit(xtrain, ytrain)

RandomForestClassifier()

In [400]:
randomforest.score(xtest, ytest)


0.519350811485643

In [835]:
df_adjust.to_csv('/Users/gavin/Documents/Metis/Coursework/Project_3/data_cleaning/df_adjust.csv')

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [30], 'max_features': [8], 'min_samples_split': [1, 5, 7, 10, 12, 15]},
    ]
forest_reg = RandomForestClassifier()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)

In [609]:
 grid_search.best_estimator_

RandomForestClassifier(max_features=8, min_samples_split=12, n_estimators=30)

In [482]:
 grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [479]:
X.head()

,over_under_line,offensive_defensive_rank_diff,offensive_offensive_rank_dif,offense_defense_passing_completion,offense_rushing_vs_defense_rushing_allowed_yards,defense_offense_rank_dif,QB_rating,target_spread,first_down_vs_first_down_allowed,first_down_allowed_vs_first_down,drives_with_score_vs_drives_against_with_score,drives_against_with_score_vs_drives_with_score,spread_to_rank_O_to_D,yards,is_favorite
0,51.0,8,2,-0.001416,0.964683,-5,101.9,-7.0,1.064220,0.915408,1.337461,0.713580,1.142857,5991,1
1,41.0,-8,-14,-0.055876,1.100013,1,76.3,-7.5,0.960245,0.873112,0.965944,0.664198,-1.066667,5688,1
2,45.5,-15,-21,0.030268,0.909334,-16,94.5,-4.0,1.051988,0.903323,1.043344,0.933333,-3.750000,5949,1
3,48.5,-18,-24,-0.102663,1.199853,-8,74.1,-1.0,0.737003,0.966767,0.758514,0.785185,-18.000000,4761,1
4,41.5,2,-4,0.018954,1.233704,2,95.4,-3.5,0.926606,0.945619,1.173375,0.674074,0.571429,5299,1


In [827]:
randomforestadj = RandomForestClassifier(max_features=4, n_estimators=30, min_samples_leaf =6, min_samples_split=7)#Min_leaf, min_sample_split

In [828]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5)

In [829]:
randomforestadj.fit(xtrain, ytrain)

RandomForestClassifier(max_features=4, min_samples_leaf=6, min_samples_split=7,
                       n_estimators=30)

In [830]:
randomforestadj.score(xtest, ytest)

0.5183520599250936

In [497]:
xtest.head()

,over_under_line,offensive_defensive_rank_diff,offensive_offensive_rank_dif,offense_defense_passing_completion,offense_rushing_vs_defense_rushing_allowed_yards,defense_offense_rank_dif,QB_rating,target_spread,first_down_vs_first_down_allowed,first_down_allowed_vs_first_down,drives_with_score_vs_drives_against_with_score,drives_against_with_score_vs_drives_with_score,spread_to_rank_O_to_D,yards,is_favorite
2183,50.5,9,0,0.169347,1.021752,-9,110.2,-5.0,1.115756,0.896254,1.357349,0.736730,1.800000,5982,1
1041,42.5,0,0,-0.058578,1.094925,0,79.5,7.0,1.052459,0.950156,0.798982,1.251592,0.000000,5359,0
2357,49.5,-3,-9,-0.025422,1.149374,-20,74.7,2.0,1.033951,1.038012,0.958457,1.150000,1.500000,5469,0
1013,44.0,-22,-22,-0.129407,1.130333,-27,83.4,1.0,0.869697,1.000000,0.780627,0.943052,22.000000,4930,0
1608,57.0,15,0,0.019604,0.914588,-15,108.0,3.5,0.938667,1.065341,1.033735,0.967366,-4.285714,6226,0


In [831]:
from sklearn.model_selection import train_test_split

def quick_test(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
    model.fit(xtrain, ytrain)
    return model.score(xtest, ytest)

def quick_test_expand(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5)
    model.fit(xtrain, ytrain)
    return (model.score(xtest, ytest), model.score(xtrain, ytrain))

def quick_test_afew_times(model, X, y, n=10):
    return np.mean([quick_test(model, X, y) for j in range(n)])

def quick_test_afew_times_view(model, X, y, n=10):
    return [quick_test_expand(model, X, y) for j in range(n)]

In [832]:
quick_test_afew_times_view(randomforest, X, y, n=10)

[(0.5370786516853933, 0.9992503748125937),
 (0.5550561797752809, 0.9992503748125937),
 (0.5318352059925093, 1.0),
 (0.552059925093633, 0.9985007496251874),
 (0.5318352059925093, 0.9985007496251874),
 (0.5580524344569289, 0.9992503748125937),
 (0.5265917602996255, 0.9992503748125937),
 (0.5258426966292135, 0.9992503748125937),
 (0.5408239700374532, 1.0),
 (0.5191011235955056, 1.0)]